In [1]:
import os, sys
import numpy as np
import torch
import espaloma as esp
import qcportal as ptl
from collections import Counter
from openff.toolkit.topology import Molecule
from openff.qcsubmit.results import BasicResultCollection
from simtk import unit
from simtk.unit import Quantity
from matplotlib import pyplot as plt

In [2]:
collection_type = "Dataset"
name = "RNA Single Point Dataset v1.0"

client = ptl.FractalClient()
collection = client.get_collection(collection_type, name)

#### check record

In [3]:
rna = client.get_collection(collection_type, name)
rna.list_records()

,driver,program,method,basis,keywords,name
0,gradient,psi4,wb97m-d3bj,def2-tzvppd,wb97m-d3bj/def2-tzvppd,WB97M-D3BJ/def2-tzvppd-wb97m-d3bj/def2-tzvppd
2,gradient,psi4,b3lyp,dzvp,default,B3LYP/dzvp-default
3,gradient,psi4,b3lyp-d3bj,dzvp,default,B3LYP-D3BJ/dzvp-default


In [4]:
#recs = collection.get_records(method='wb97m-d3bj', basis='def2-tzvppd', program='psi4', keywords='wb97m-d3bj/def2-tzvppd')
recs = collection.get_records(method='b3lyp-d3bj', basis='dzvp', program='psi4', keywords='default')
print("length of dataframe", len(recs), "\n", "No. of records in each: ",  len(recs[0]), len(recs[1]))

length of dataframe 2 
 No. of records in each:  4489 4489


#### inspect

In [5]:
# Sample dictionary of a record, and the properties that can be accessed through this
d3bj_correction_dataframe = recs[0]
# first record 
#print(d3bj_correction_dataframe.iloc[0].record.dict())

In [6]:
# Sample dictionary of a record, and the properties that can be accessed through this
b3lyp_dataframe = recs[1]
# first record 
#print(b3lyp_dataframe.iloc[0].record.dict())

In [7]:
#b3lyp_dataframe.record[0].dict()

In [8]:
#d3bj_correction_dataframe.record[0].dict()

#### combine two energies

In [9]:
# Some calculations failed due to SCF convergence errors, so excluding those by checking the record status of b3lyp calculation
for i in range(len(recs[0])):
    if recs[1].iloc[i].record.status == 'COMPLETE':
        #print("B3LYP + D3BJ energy of ", recs[1].iloc[i].record.id, recs[1].iloc[i].name, recs[1].iloc[i].record.properties.return_energy + recs[0].iloc[i].record.properties.return_energy)
        pass

In [10]:
print("B3LYP + D3BJ energy of ", recs[1].iloc[i].record.id, recs[1].iloc[i].name, recs[1].iloc[i].record.properties.return_energy + recs[0].iloc[i].record.properties.return_energy)

B3LYP + D3BJ energy of  109520662 O=c1ccn([C@@H]2O[C@H](COP(=O)([O-])O[C@@H]3[C@@H](COP(=O)([O-])O[C@@H]4[C@@H](CO)O[C@@H](n5ccc(=O)[nH]c5=O)[C@@H]4O)O[C@@H](n4ccc(=O)[nH]c4=O)[C@@H]3O)[C@@H](O)[C@H]2O)c(=O)[nH]1-9 -3715.1842143644626


#### combine two gradient

In [11]:
#recs[1].iloc[i].record.dict()

In [12]:
#assert recs[0].iloc[-1].record.properties.scf_total_energy == recs[0].iloc[-1].record.properties.return_energy
print(recs[0].iloc[-1].record.properties.scf_total_energy, recs[0].iloc[-1].record.properties.return_energy)

None -0.20735354


In [13]:
assert recs[1].iloc[-1].record.properties.scf_total_energy == recs[1].iloc[-1].record.properties.return_energy

In [14]:
return_result = recs[1].iloc[-1].record.return_result
current_grad = recs[1].iloc[-1].record.extras['qcvars']['CURRENT GRADIENT']
dft_total_grad = recs[1].iloc[-1].record.extras['qcvars']['DFT TOTAL GRADIENT']
scf_total_grad = recs[1].iloc[-1].record.extras['qcvars']['SCF TOTAL GRADIENT']

In [15]:
#(return_result - current_grad).flatten()

In [16]:
#(return_result - dft_total_grad).flatten()

In [17]:
#(return_result - scf_total_grad).flatten()

In [18]:
# Some calculations failed due to SCF convergence errors, so excluding those by checking the record status of b3lyp calculation
#print("B3LYP + D3BJ energy of ", recs[1].iloc[-1].record.id, recs[-1].iloc[i].name, recs[1].iloc[-1].record.return_result + recs[0].iloc[-1].record.return_result)

In [19]:
#recs[1].iloc[-1].record.return_result[:3]

In [20]:
#recs[0].iloc[-1].record.return_result[:3]

#### convert mol to graph

In [21]:
i = -1
mol = client.query_molecules(recs[0].iloc[i].record.molecule)[0]
mol

NGLWidget()

In [22]:
offmol = Molecule.from_qcschema(mol, allow_undefined_stereo=True)   # convert to OpenFF Molecule object
offmol.compute_partial_charges_am1bcc()   # https://docs.openforcefield.org/projects/toolkit/en/0.9.2/api/generated/openff.toolkit.topology.Molecule.html

Warning (not error because allow_undefined_stereo=True): OEMol has unspecified stereochemistry. oemol.GetTitle(): 
Problematic atoms are:
Atom atomic num: 15, name: , idx: 28, aromatic: False, chiral: True with bonds:
bond order: 1, chiral: False to atom atomic num: 8, name: , idx: 27, aromatic: False, chiral: False
bond order: 1, chiral: False to atom atomic num: 8, name: , idx: 29, aromatic: False, chiral: False
bond order: 2, chiral: False to atom atomic num: 8, name: , idx: 30, aromatic: False, chiral: False
bond order: 1, chiral: False to atom atomic num: 8, name: , idx: 31, aromatic: False, chiral: False
Atom atomic num: 15, name: , idx: 58, aromatic: False, chiral: True with bonds:
bond order: 1, chiral: False to atom atomic num: 8, name: , idx: 57, aromatic: False, chiral: False
bond order: 1, chiral: False to atom atomic num: 8, name: , idx: 59, aromatic: False, chiral: False
bond order: 2, chiral: False to atom atomic num: 8, name: , idx: 60, aromatic: False, chiral: False
bo

In [23]:
torch.tensor(offmol.partial_charges.value_in_unit(esp.units.CHARGE_UNIT))

tensor([-0.5622,  0.3857,  0.1125,  0.0082,  0.0082,  0.0966,  0.1489, -0.4368,
         0.1804,  0.0983, -0.5822,  0.1144,  0.0624, -0.5808,  0.4376,  0.2652,
         0.1247, -0.3635,  0.8083, -0.6184, -0.5942,  0.3410,  0.7215, -0.6601,
        -0.3701,  0.1633,  0.0853,  0.1740,  1.4328, -0.8183, -0.8183, -0.5609,
         0.1499,  0.0346,  0.0346,  0.1110,  0.1314, -0.4176,  0.1607,  0.0687,
        -0.5831,  0.1036,  0.0832, -0.5977,  0.4264,  0.2630,  0.1091, -0.3645,
         0.8058, -0.6565, -0.5970,  0.3346,  0.7178, -0.6647, -0.3537,  0.1806,
         0.0860,  0.2170,  1.4255, -0.8063, -0.8063, -0.5662,  0.1594,  0.0361,
         0.0361,  0.1313,  0.1017, -0.4254,  0.1320,  0.1092, -0.5925,  0.4181,
         0.0686,  0.0772, -0.5910,  0.4241,  0.2690,  0.1032, -0.3816,  0.8052,
        -0.6755, -0.5879,  0.3422,  0.7183, -0.6373, -0.3382,  0.1819,  0.0825,
         0.1998], dtype=torch.float64)

In [24]:
_energy1 = recs[0].iloc[i].record.properties.return_energy
_energy2 = recs[1].iloc[i].record.properties.return_energy

_grad1 = recs[0].iloc[i].record.return_result
_grad2 = recs[0].iloc[i].record.return_result

In [25]:
energy = _energy1 + _energy2
grad = _grad1 + _grad2
print(energy)
print(grad[:5])

-3715.1842143644626
[[ 0.00076727  0.00087557 -0.00136133]
 [ 0.00021946  0.00039229 -0.00060083]
 [ 0.00147035  0.00101746 -0.00067559]
 [ 0.00060044  0.00037367 -0.00036135]
 [ 0.00054612  0.00046903 -0.00011695]]


In [26]:
g = esp.Graph(offmol)
    
# energy is already hartree
e = torch.tensor(
    [
        Quantity(
            energy,
            esp.units.HARTREE_PER_PARTICLE,
        ).value_in_unit(esp.units.ENERGY_UNIT)
    ],
    dtype=torch.get_default_dtype(),
)[None, :]

xyz = torch.tensor(
    np.stack(
        [
            Quantity(
                mol.geometry,
                unit.bohr,
            ).value_in_unit(esp.units.DISTANCE_UNIT)
        ],
        axis=1,
    ),
    requires_grad=True,
    dtype=torch.get_default_dtype(),
)

u = torch.stack(
    [
        torch.tensor(
            Quantity(
                grad,
                esp.units.HARTREE_PER_PARTICLE / unit.bohr,
            ).value_in_unit(esp.units.FORCE_UNIT),
            dtype=torch.get_default_dtype(),
        )
    ],
    dim=1,
)

c = torch.tensor(offmol.partial_charges.value_in_unit(esp.units.CHARGE_UNIT), dtype=torch.get_default_dtype(),).unsqueeze(-1)

/Users/takabak/mambaforge/envs/espaloma/lib/python3.9/site-packages/dgl-0.9.0-py3.9-macosx-11.0-arm64.egg/dgl/heterograph.py:72: DGLWarning: Recommend creating graphs by `dgl.graph(data)` instead of `dgl.DGLGraph(data)`.
  dgl_warning('Recommend creating graphs by `dgl.graph(data)`'


In [27]:
print(e.shape, xyz.shape, u.shape, c.shape)

torch.Size([1, 1]) torch.Size([89, 1, 3]) torch.Size([89, 1, 3]) torch.Size([89, 1])


#### save as pickle

In [28]:
# save as pickle object
import pickle
with open('offmol.pkl', 'wb') as pkl:
     pickle.dump(offmol, pkl, protocol=4)

In [29]:
with open('offmol.pkl', 'rb') as db:
    offmol_pkl = pickle.load(db)

In [30]:
offmol_pkl

NGLWidget()

In [31]:
offmol_pkl.partial_charges - offmol.partial_charges

Quantity(value=array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0.]), unit=elementary charge)